###  Chetan Arjun Nakhate (1002157911)
### Neha More (1002060504)
### Harsha Vardhan Nunna (1002159569)

# Assignment 1 - Exploratory Data Analysis

# 🌟 Programming Assignment Details 🌟

Hey there, amazing data miners! 😊 Before you dive into the exciting world of data, here are a few friendly pointers for your upcoming programming assignment:

1. **Resource Usage** 📚💻🤖: We totally encourage the use of various resources like books, internet sources, or even AI tools to aid your learning and assignment completion. It's all part of the learning journey! Just remember, if you do use any of these helpful resources, please give them a little shoutout (aka cite them) in the same cell where you've used them. It's all about giving credit where it's due!

2. **Dataset File Names** 📁: You'll find some dataset files as part of this assignment. They're like little treasure chests of information, waiting to be unlocked by you! However, please make sure not to change their names. Think of their names as their unique identities - it helps us, and you, to keep track of them easily.

3. **Image Analysis Submission** 📊🖼️: As you explore and analyze images, we'd love to see your findings! Please include all the image analyses in your submission folder. And here's a fun twist – let's name each image file according to the task it represents (like 'Task3B', 'Task3C', etc.). It's a great way to stay organized and it adds a personal touch to your work.

We're super excited to see the wonderful insights you'll uncover! Dive in, enjoy, and remember – we're here to support you every step of the way. Happy data mining! 🌟🛠️💡

## Instructions

For the First Assignment, we are going to be Performing Exploratory Data Analysis Using Python, R and LLM

1. There are set of questions that may be similar for python and R .
2. The dataset used for Python and R are different.
3. For Task involving R use a different R notebook.
3. Each Task is presenting a challenge.
4. When using LLM make sure that you cite the tool and it corresponding prompts.

## Task 2: R (40 Points)

Run this cell by default to load the dataset.

The data contains intakes and outcomes of animals entering the Animal Center from the beginning of October 2013 to the present day. 

In [3]:
# Required libraries
library(readr)
library(dplyr)
library(ggplot2)
library(tidyr)
library(lubridate)
library(patchwork) # For combining plots
library(gridExtra)

ERROR: Error in library(readr): there is no package called 'readr'


### Task 1A: 2 points
Display all the details about the dataset

In [ ]:
# Load the dataset
dataset <- read_csv("Animal_Center_Intakes.csv")
print(colnames(dataset))

### Task 1B: 4 points
1. Check for null values within each column.
2. Fill the values for more than 100 missing columns with certain label such as Not available, Not Name, etc. Drop the rows for the rest missing columns.
3. Convert the date columns into appropriate type.

In [ ]:
# 1. Check for null values within each column
null_values <- colSums(is.na(dataset))
print(null_values)

In [ ]:
# 2. Fill values for columns with more than 100 missing entries and drop rows for others
columns_to_check <- colnames(dataset)
columns_with_many_missing <- sapply(columns_to_check, function(col) sum(is.na(dataset[[col]])) > 100)

# Drop rows with missing values only for columns with more than 100 missing entries
rows_to_keep <- complete.cases(dataset[, columns_with_many_missing])

# Drop rows with missing values for other columns
rows_to_keep <- rows_to_keep & complete.cases(dataset[, !columns_with_many_missing])
                                                      
# Filter dataset
dataset <- dataset[rows_to_keep, ]
print(summary(dataset))

In [ ]:

# Assuming 'IntakeDate' is the date column
dataset$DateTime <- as.Date(dataset$DateTime, format = "%Y-%m-%d")

### Task 1C: 4 points
1. Plot the Age Intake column and observe the distribution.
2. Group certain values into categories,
- Less than 1 week, Less than 1 month, Less than 1 year
- Drop inappropriate negative valued years rows.
3. Plot the Column to again see the new distributuions.

Tip: consider 0 years as less than 1 year.

In [ ]:
# Ensure 'Age upon Intake' column exists in dataset
if (!"Age upon Intake" %in% names(dataset)) {
  stop("'Age upon Intake' column not found in the dataset.")
}


# Convert "Age upon Intake" to numeric days
dataset$Age_Days <- sapply(as.character(dataset$`Age upon Intake`), function(x) {
  parts <- strsplit(x, " ")[[1]]
  if(length(parts) < 2) return(NA)  # Ensure there are two parts to prevent errors
  quantity <- as.numeric(parts[1])
  unit <- parts[2]
  if(unit %in% c("day", "days")) {
    return(quantity)
  } else if(unit %in% c("week", "weeks")) {
    return(quantity * 7)
  } else if(unit %in% c("month", "months")) {
    return(quantity * 30)  # Approximation of a month
  } else if(unit %in% c("year", "years")) {
    return(quantity * 365)  # Approximation of a year
  } else {
    return(NA)  # In case of an unexpected format
  }
})

# Filter out negative or NA values in 'Age_Days'
dataset <- dataset[!is.na(dataset$Age_Days) & dataset$Age_Days >= 0, ]




In [ ]:
# Categorize the 'Age_Days' into specified groups
dataset$Age_Category <- cut(dataset$Age_Days,
                            breaks = c(-Inf, 7, 30, 365, Inf),
                            labels = c("Less than 1 week", "Less than 1 month", "Less than 1 year", "1 year or more"))

In [ ]:

# Plot the distribution of age categories
ggplot(dataset, aes(x = Age_Category)) +
  geom_bar(fill = "blue") +
  theme_minimal() +
  xlab("Age Category") +
  ylab("Count") +
  ggtitle("Distribution of Age Categories After Categorization")

### Task 1D: 2 points
Print Top 15 locations where animals were found.

In [ ]:
# Group by 'Found Location', count, and then arrange in descending order
top_locations <- dataset %>%
  group_by(`Found Location`) %>%
  summarise(Count = n()) %>%
  arrange(desc(Count)) %>%
  top_n(15, Count)

# Print the top 15 locations
print(top_locations)

### Task 1E: 2 points
Print Top 10 breeds with animal type that were stray and were found in between 15 to 20 years old.

In [ ]:
dataset$Age_Years <- dataset$Age_Days / 365

# Filter for stray animals found between 15 to 20 years old
stray_15_20 <- dataset %>%
  filter(`Intake Type` == "Stray" & Age_Years >= 15 & Age_Years <= 20) %>%
  group_by(Breed, `Animal Type`) %>%
  summarise(Count = n()) %>%
  arrange(desc(Count)) %>%
  top_n(10, Count)

# Print the top 10 breeds with animal type
print(stray_15_20)

### Task 1F: 4 points
Display horizontal bar graph for Top 10 Locations where animals were found at night time that were sick.

Note: night time = 7pm to 12am

In [ ]:
df <- read.csv("Animal_Center_Intakes.csv")

# Convert 'DateTime' column to proper datetime format
df$DateTime <- as.POSIXct(df$DateTime, format = "%m/%d/%Y %I:%M:%S %p")

# Filter the dataset for entries where animals were found at night time and were sick
night_sick <- df %>%
  filter(hour(DateTime) >= 19 & hour(DateTime) < 24 & `Intake.Condition` == "Sick")

# Count occurrences of each location
location_counts <- table(night_sick$`Found.Location`)

# Sort the locations by count and select the top 10
top_locations <- names(sort(location_counts, decreasing = TRUE)[1:10])

# Filter the dataset to include only the top 10 locations
top_locations_data <- night_sick %>%
  filter(`Found.Location` %in% top_locations)

# Plot horizontal bar graph
ggplot(top_locations_data, aes(y = `Found.Location`)) +
  geom_bar() +
  ggtitle("Top 10 Locations where Sick Animals were Found at Night Time") +
  xlab("Number of Sick Animals") +
  ylab("Location") +
  theme_minimal() +
  theme(axis.text.y = element_text(size = 10))



### Task 1G: 3 points

Display a pie chart for cat conditions when they were found at '4434 Frontier Trl in Austin (TX)'.

In [ ]:
# Read the dataset
df <- read_csv("Animal_Center_Intakes.csv")

# Filter the dataset for cats found at '4434 Frontier Trl in Austin (TX)'
cats_at_location <- df %>%
  filter(`Found Location` == '4434 Frontier Trl in Austin (TX)' & `Animal Type` == 'Cat')

# Count occurrences of each condition
condition_counts <- table(cats_at_location$`Intake Condition`)

# Plot pie chart
pie(condition_counts, 
    labels = paste(names(condition_counts), ": ", condition_counts), 
    main = "Cat Conditions at 4434 Frontier Trl in Austin (TX)")


### Task 1H: 4 points
Display a bar chart for Top 10 colors of animal that were surrendered.

In [ ]:
# Filter for surrendered animals
surrendered_animals <- dataset %>%
  filter( `Intake Type` == "Surrender")

# Count the occurrences of each color and select the top 10
top_colors <- surrendered_animals %>%
  group_by(Color) %>%
  summarise(Count = n()) %>%
  arrange(desc(Count)) %>%
  top_n(10, Count)

# Plot the bar chart
ggplot(top_colors, aes(x = reorder(Color, Count), y = Count)) +
  geom_bar(stat = "identity", fill = "steelblue") +
  theme_minimal() +
  labs(title = "Top 10 Colors of Surrendered Animals",
       x = "Color",
       y = "Count") +
  coord_flip() # Flip coordinates for horizontal bar chart


### Task 1I: 6 points

Display a 2 by 2 graphs where spots are defined as
1. Pie chart for intake type.
2. Pie chart for Intake Condition
3. Pie chart for Animal Type.
4. Horizontal Distribution for Intake condition for Dogs.

In [ ]:
df <- read.csv("Animal_Center_Intakes.csv")
print(colnames(df))
# Load required library
library(ggplot2)

# Create a 2 by 2 grid layout
par(mfrow = c(2, 2))

# Plot 1: Pie chart for Intake Type
intake_type_counts <- table(df$`Intake.Type`)
if (length(intake_type_counts) > 0) {
  pie(intake_type_counts, main = "Intake Type")
} else {
  message("No data available for Intake Type")
}

# Plot 2: Pie chart for Intake Condition
intake_condition_counts <- table(df$`Intake.Condition`)
if (length(intake_condition_counts) > 0) {
  pie(intake_condition_counts, main = "Intake Condition")
} else {
  message("No data available for Intake Condition")
}

# Plot 3: Pie chart for Animal Type
animal_type_counts <- table(df$`Animal.Type`)
if (length(animal_type_counts) > 0) {
  pie(animal_type_counts, main = "Animal Type")
} else {
  message("No data available for Animal Type")
}

# Plot 4: Horizontal bar plot for Intake Condition for Dogs
dog_intake_condition_counts <- table(subset(df, `Animal.Type` == "Dog")$`Intake.Condition`)
if (length(dog_intake_condition_counts) > 0) {
  barplot(dog_intake_condition_counts, horiz = TRUE, main = "Intake Condition for Dogs")
} else {
  message("No data available for Intake Condition for Dogs")
}


### Task 1J: 3 points
Print Top 10 Breeds of animal who were less than a year old when they were taken in along with what type of animal they were.

In [ ]:
# Read the dataset
df <- read.csv("Animal_Center_Intakes.csv")

# Convert Age upon Intake to numeric
df$Age.numeric <- as.numeric(sub(" .*", "", df$Age.upon.Intake))

# Filter for animals less than a year old
filtered_df <- df %>% filter(Age.numeric < 1)

# Count occurrences of each breed for animals less than a year old
top_breeds <- filtered_df %>%
  group_by(`Animal.Type`, Breed) %>%
  summarise(count = n()) %>%
  arrange(`Animal.Type`, desc(count)) %>%
  group_by(`Animal.Type`) %>%
  slice_max(order_by = count, n = 10)

# Print top 10 breeds along with animal type
print(breed_counts)




### Task 1K: 6 points
1. Print the count of distinct sex intake from 2015 to 2020.
2. Display a side by side bar graph this.

In [ ]:

df <- read.csv("Animal_Center_Intakes.csv")

# Convert DateTime column to date format
df$DateTime <- as.POSIXct(df$DateTime, format = "%m/%d/%Y %I:%M:%S %p")

# Extract year from DateTime column
df$Year <- lubridate::year(df$DateTime)

# Filter data for the years 2015 to 2020
filtered_df <- df %>% filter(Year >= 2015 & Year <= 2020)

# Count distinct sex intake
sex_intake_count <- filtered_df %>% group_by(Sex.upon.Intake) %>% summarise(count = n_distinct(Animal.ID))

# Print count of distinct sex intake
print(sex_intake_count)



In [ ]:
# Plot side by side bar graph
ggplot(sex_intake_count, aes(x = Sex.upon.Intake, y = count, fill = Sex.upon.Intake)) +
  geom_bar(stat = "identity") +
  labs(title = "Count of Distinct Sex Intake from 2015 to 2020", x = "Sex upon Intake", y = "Count") +
  theme_minimal() +
  theme(legend.position = "none")